In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def trapezoidal_wfm(t, T, Trise=1e-15, Tfall=1e-15, D=0.5, A=1, A0=0, DEBUG=False):
    """Generate a trapezoidal waveform.  This method is far from unique, and is
    arguably less readable than if I had generated the same waveform with a 
    loop iteration for each signal period."""
    r0 = np.floor(t/T)
    r = t/T - r0 # Creates a periodic array ranging from 0 - 1 based on the 
    # period of the waveform.
    
    # Rising edge
    r_rise = Trise/T
    rslope_rise = 2*A/r_rise
    # Stay at A until hitting the falling edge, defined as r_mid below.

    # Then, fall back to -A 
    # Falling edge
    r_fall = Tfall/T
    rslope_fall = 2*A/r_fall
    # Stay at -A for the duration of the cycle (until r = 0 again)    
    
    # Midpoint, defined by the duty cycle, rise, and fall points.
    r_mid = r_rise/2 + D - r_fall/2 # r_mid is calculated later b/c of dependence on r_fall.

    if DEBUG:
        print(0, r_rise, r_mid, r_mid+r_fall, 1)

    # Each line of the following expression represents a phase in the waveform.
    # Line 1: Rising Edge
    # Line 2: High Level
    # Line 3: Falling Edge
    # Line 4: Low Level
    # Line 5: DC offset
    # The nature of this expression is that it is inherently periodic.
    x = ((r >= 0)&(r < r_rise))*(r*rslope_rise - A) \
        + ((r >= r_rise)&(r < r_mid))*A \
        + ((r >= r_mid)&(r < r_fall+r_mid))*(A - (r-r_mid)*rslope_fall) \
        + ((r >= r_fall+r_mid)&(r < 1))*(-A)\
        + A0
        
    return x

# Trapezoidal Fourier Series Coefficient Calculations
def trapezoidal_fs_coeff(T, n, Trise=1e-15, Tfall=1e-15, D=0.5, A=1, A0=0, print_coeff=False, zero_tol=1e-18):
    """
    $$D_n = \frac{1}{T_0} \int_0^{T_0} x(t) e^{-j 2\pi n f_0 t} dt$$
    
    $$x(t) = \sum_{n=-\infty}^\infty D_n e^{j 2\pi n f_0 t}$$
    """
    # The DC term D_0 = A0.
    if n == 0:
        D0 = A0 + A*(2*D-1)
        if print_coeff:
            print("D[%d] = %.3g at %.1f deg" % (n, np.abs(D0), (180*np.angle(D0)/np.pi)))
        return D0
    
    # The remainder of this function assume n > 0.  
    def const_term_integration(T, n, a, b):
        """
        $$I_0 = \int_a^b e^{-j 2\pi n f_0 t} dt$$
        """
        f0 = 1./T
        alpha = -2j*np.pi*n*f0
        I = 1./alpha * (np.e**(alpha*b) - np.e**(alpha*a))
        return I
    
    def ramp_term_integration(T, n, a, b):
        """
        $$I_1 = \int_a^b t e^{-j 2\pi n f_0 t} dt$$
        """
        f0 = 1./T
        alpha = -2j*np.pi*n*f0
        I = 1./alpha * (b*np.e**(alpha*b) - a*np.e**(alpha*a)) \
            - 1./(alpha*alpha) * (np.e**(alpha*b) - np.e**(alpha*a))
        return I
    
    Tpulse = D*T
    t1 = 0
    t2 = Trise
    t3 = Trise/2 + Tpulse - Tfall/2
    t4 = t3 + Tfall
    t5 = T

    Irise = (2*A/Trise) * ramp_term_integration(T, n, t1, t2) \
        + (A0 - A) * const_term_integration(T, n, t1, t2)
    Ihigh = (A0 + A) * const_term_integration(T, n, t2, t3)
    Ifall = (A0 + A + 2*A*t3/Tfall) * const_term_integration(T, n, t3, t4) \
        - (2*A/Tfall) * ramp_term_integration(T, n, t3, t4)
    Ilow = (A0 - A) * const_term_integration(T, n, t4, t5)
    
    Dn = (Irise + Ihigh + Ifall + Ilow)/T
    if np.abs(Dn) < zero_tol:
        Dn = 0. + 0j
    
    if print_coeff:
        print("D[%d] = %.3g at %.1f deg" % (n, np.abs(Dn), (180*np.angle(Dn)/np.pi)))
    return Dn

def trapezoidal_fs_approx(t, N, T, Trise=1e-15, Tfall=1e-15, D=0.5, A=1, A0=0, print_coeff=False, zero_tol=1e-18):
    """
    Approximate the time-domain waveform with an incomplete number of Fourier terms.
    """
    xapprox = np.zeros(t.shape, dtype=np.complex128)
    
    for n in range(-N, N+1):
        Dn = trapezoidal_fs_coeff(T, n, Trise, Tfall, D, A, A0, print_coeff=print_coeff, zero_tol=zero_tol)
        exp = np.e**(2j*np.pi*n*t/T)
        xapprox += Dn*exp
    
    return np.real(xapprox)

def trapezoidal_fourier_series_spectrum(N, T, Trise=1e-15, Tfall=1e-15, D=0.5, A=1, A0=0, print_coeff=False, zero_tol=1e-18):
    """
    Return a frequency vector and coefficients a trapezoidal waveform.
    """    
    nv = np.arange(-N, N+1)
    fv = nv/T # f0 = 1/T ==> fn = n f0
    Dv = []
    for n in nv:
        Dn = trapezoidal_fs_coeff(T, n, Trise, Tfall, D, A, A0, print_coeff=print_coeff, zero_tol=zero_tol)
        Dv.append(Dn)    
    return fv, np.array(Dv)

def trapezoidal_bode_plot(f, T, Trise=1e-15, D=0.5, A=1, ReturnCutoff=False):
  """Calculate and return a Bode plot envelope function for the magnitude spectrum of
  the trapezoidal wave described by the given parameters.  Use the frequency vector f
  and return a magnitude spectrum X."""
  Tpulse = D*T
  f1 = 1./(np.pi * Tpulse)
  f2 = 1./(np.pi * Trise)

  Xm = (A*D) \
    * np.array([1 if f_ < f1 else f1/f_ for f_ in f]) \
    * np.array([1 if f_ < f2 else f2/f_ for f_ in f])

  if ReturnCutoff:
    return Xm, f1, f2
  else:
    return Xm

def nearest_index(a, v):
    """
    Act similar to numpy.searchsorted, but find the nearest point
    rather than using a before or after approach.

    a - is the full vector
    v - is the value to search on.
    """
    if type(v) in (list, tuple, np.ndarray):
        inearest = []
        for v_ in v:
            inearest.append(nearest_index(a, v_))
        return inearest
    else:
        # The real calculation.
        return np.argmin(np.abs(np.array(a)-v))


def marker_fcn(a, v, y):
    """
    Use nearest_index to determine the closest indices.  Then, output
    the y values associated with these indices like a marker on an
    oscilloscope or spectrum analyzer.
    """
    il = nearest_index(a, v)
    return np.array(y)[il]

In [3]:
# Problem 1
A = 2.5
A0 = 2.5
T = 1./5e6
D = 0.3
Tpulse = D*T
Trise = 15e-9
Tfall = 15e-9

# Harmonic Frequencies
fharm = np.arange(1,6)/T

# dBuV Calculation
yfcn = lambda y_ : 20*np.log10(np.abs(y_)/1e-6)

# Bode
fv = np.logspace(6, 9, 10001)
Yenv, f1, f2 = trapezoidal_bode_plot(fv, T, Trise, D, A+A0, ReturnCutoff=True)
Yenv *= 2 # Doubling here to go single-sided.
ii1, ii2 = nearest_index(fv, [f1, f2])
Yenv1, Yenv2 = Yenv[ii1], Yenv[ii2]

iv_harm = nearest_index(fv, fharm)
print(', '.join(['%.3f' % f_ for f_ in fv[iv_harm]/1e6]))
print(', '.join(['%.3f' % yfcn(y_) for y_ in Yenv[iv_harm]]))

# FS Calculation
f_fs, Y_fs = trapezoidal_fourier_series_spectrum(5, T, Trise=Trise, Tfall=Tfall, D=D, A=A, A0=A0, print_coeff=False, zero_tol=1e-18)
#print(Y_fs)
#Y_fs_singlesided = Y_fs[:6:-1] + Y_fs[7::]
Y_fs *= 2
#print(yfcn(Y_fs[0:6:-1]))
print(yfcn(Y_fs[6:]))
#print(yfcn(Y_fs))
#print(yfcn(Y_fs_singlesided))

# Plot
fig, ax = plt.subplots(1,1)
ax.set_xlim(1e6, 1e9)
ax.set_ylim(50,140)
ymin, ymax = ax.get_ylim()
print(yfcn(Yenv1), yfcn(Yenv2))

ax.semilogx(fv, yfcn(Yenv), label='Envelope', color='k')
ax.axvline(f1, ymax=(yfcn(Yenv1)-ymin)/(ymax-ymin), color='k', linestyle='--')
ax.axvline(f2, ymax=(yfcn(Yenv2)-ymin)/(ymax-ymin), color='k', linestyle='--')

ax.semilogx(f_fs, yfcn(Y_fs), label='Exact', color='C1', linestyle='none', marker='o')


#ax.set_xticklabels([])
#ax.set_yticklabels([])
ax.grid(True)
ax.set_xlabel('Frequency')
ax.set_ylabel('Magnitude (dB$\mu$V)')
ax.text(1.5e6, 133, '$0\ dB/dec$')
ax.text(1.e7, 125, '$-20\ dB/dec$')
ax.text(1e8, 95, '$-40\ dB/dec$')
ax.text(1e10, 120, ('$f_1 = %.3f$ MHz' % (f1/1e6)))
ax.text(1e10, 115, ('$f_2 = %.3f$ MHz' % (f2/1e6)))

fig.savefig('figures/problem01_solution.pdf')

5.000, 9.998, 14.997, 20.003, 25.003
129.542, 124.038, 120.516, 118.014, 114.652
[128.13563778 123.27664181 109.57846524 112.07390677 113.96628657]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

129.5404026328377 117.49557999784139


In [4]:
# Problem 4
A = 8
T = 1./354e6
Trise = 100e-12
Tfall = Trise
D = 0.45
Tpulse = D*T

fv = np.logspace(7, 11, 10001) # 1 Hz to 100 kHz
Yenv, f1, f2 = trapezoidal_bode_plot(fv, T, Trise, D, A, ReturnCutoff=True)
ii1, ii2 = np.searchsorted(fv, [f1, f2])
Yenv1, Yenv2 = Yenv[ii1], Yenv[ii2]


fig, ax = plt.subplots(1,1)
ax.set_xlim(1e7, 1e11)
ax.set_ylim(50,140)
ymin, ymax = ax.get_ylim()
yfcn = lambda y_ : 20*np.log10(np.abs(y_)/1e-6)
print(yfcn(Yenv1), yfcn(Yenv2))
ax.semilogx(fv, yfcn(Yenv), label='Envelope', color='k')
ax.axvline(f1, ymax=(yfcn(Yenv1)-ymin)/(ymax-ymin), color='k', linestyle='--')
ax.axvline(f2, ymax=(yfcn(Yenv2)-ymin)/(ymax-ymin), color='k', linestyle='--')

#ax.set_xticklabels([])
#ax.set_yticklabels([])
ax.grid(True)
ax.set_xlabel('Frequency')
ax.set_ylabel('Envelope Magnitude (dB$\mu$V)')
ax.text(4e7, 133, '$0\ dB/dec$')
ax.text(4e8, 110, '$-20\ dB/dec$')
ax.text(1e10, 95, '$-40\ dB/dec$')
ax.text(1e10, 120, ('$f_1 = %.3f$ MHz' % (f1/1e6)))
ax.text(1e10, 115, ('$f_2 = %.3f$ GHz' % (f2/1e9)))

fig.savefig('figures/problem04_solution.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

131.12286752647196 109.0278700725893


In [5]:
# Problem 4
A = 8
T = 1./354e6
Trise = 100e-12
Tfall = Trise
D = 0.45
Tpulse = D*T

fv = np.logspace(7, 11, 10001) # 1 Hz to 100 kHz
Yenv, f1, f2 = trapezoidal_bode_plot(fv, T, Trise, D, A, ReturnCutoff=True)
ii1, ii2 = np.searchsorted(fv, [f1, f2])
Yenv1, Yenv2 = Yenv[ii1], Yenv[ii2]


fig, ax = plt.subplots(1,1)
ax.set_xlim(1e7, 1e11)
ax.set_ylim(50,140)
ymin, ymax = ax.get_ylim()
yfcn = lambda y_ : 20*np.log10(np.abs(y_)/1e-6)
print(yfcn(Yenv1), yfcn(Yenv2))
ax.semilogx(fv, yfcn(Yenv), label='Envelope', color='k')
ax.axvline(f1, ymax=(yfcn(Yenv1)-ymin)/(ymax-ymin), color='k', linestyle='--')
ax.axvline(f2, ymax=(yfcn(Yenv2)-ymin)/(ymax-ymin), color='k', linestyle='--')

#ax.set_xticklabels([])
#ax.set_yticklabels([])
ax.grid(True)
ax.set_xlabel('Frequency')
ax.set_ylabel('Envelope Magnitude (dB$\mu$V)')
ax.text(4e7, 133, '$0\ dB/dec$')
ax.text(4e8, 110, '$-20\ dB/dec$')
ax.text(1e10, 95, '$-40\ dB/dec$')
ax.text(1e10, 120, ('$f_1 = %.3f$ MHz' % (f1/1e6)))
ax.text(1e10, 115, ('$f_2 = %.3f$ GHz' % (f2/1e9)))

fig.savefig('figures/problem04_solution.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

131.12286752647196 109.0278700725893
